In [16]:
# Imports
from azure.common.credentials import ServicePrincipalCredentials
from azure.identity import ClientSecretCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.storage import StorageManagementClient
from azureml.core import Workspace
# from azure.ai.ml.entities import Workspace
import yaml

In [3]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

subscription_id = config.get("subscription_id")
client_id = config.get("client_id")
secret = config.get("secret")
tenant_id = config.get("tenant_id")


In [4]:
credential = ClientSecretCredential(
    client_id=client_id,
    client_secret=secret,
    tenant_id=tenant_id
)

from azureml.core.authentication import ServicePrincipalAuthentication

credential_aml = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=client_id,
    service_principal_password=secret)

# credential_sp = ServicePrincipalCredentials(
#     client_id=client_id,
#     secret=secret,
#     tenant=tenant_id
# )

In [5]:
# Configuration Parameters
resource_group_name = 'AI_IC_NAM_GenAI-Template-2'
location = 'eastus'
vnet_name = 'MyVNet'
subnet_name = 'MySubnet'
aml_workspace_name = 'genai-test-workspace'
storage_account_name = 'genaitemplate1storage'
openai_account_name = 'genai-test-openai'

resource_client = ResourceManagementClient(credential, subscription_id)
network_client = NetworkManagementClient(credential, subscription_id)
storage_client = StorageManagementClient(credential, subscription_id)

In [6]:
# Create Resource Group
resource_client.resource_groups.create_or_update(resource_group_name, {'location': location})

In [7]:
# Create Virtual Network and Subnet
vnet_params = {
    'location': location,
    'address_space': {
        'address_prefixes': ['10.0.0.0/16']
    }
}
network_client.virtual_networks.begin_create_or_update(resource_group_name, vnet_name, vnet_params)

subnet_params = {
    'address_prefix': '10.0.0.0/24',
    'service_endpoints': [{'service': 'Microsoft.CognitiveServices'}]
}
network_client.subnets.begin_create_or_update(resource_group_name, vnet_name, subnet_name, subnet_params)

# Add Network Rule for Subnet
subnet = network_client.subnets.get(resource_group_name, vnet_name, subnet_name)
subnet_id = subnet.id

# Work in Progress

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.network import NetworkManagementClient

subscription_id = "your-subscription-id"
resource_group = "your-resource-group"
vnet_name = "your-vnet-name"
nsg_name = "your-nsg-name"

# Authenticate
credential = DefaultAzureCredential()
network_client = NetworkManagementClient(credential, subscription_id)

# Create NSG
nsg_params = {
    "location": "your-location",
}
network_client.network_security_groups.begin_create_or_update(resource_group, nsg_name, nsg_params).result()

# Link NSG to VNet
subnet = network_client.subnets.begin_get(resource_group, vnet_name, "your-subnet-name").result()
subnet.network_security_group = {
    'id': f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Network/networkSecurityGroups/{nsg_name}"
}
network_client.subnets.begin_create_or_update(resource_group, vnet_name, "your-subnet-name", subnet).result()

# Update inbound rules to allow specific traffic (e.g., SSH port 22)
security_rule = {
    "name": "Allow_SSH",
    "protocol": "Tcp",
    "source_port_range": "*",
    "destination_port_range": "22",
    "source_address_prefix": "*",
    "destination_address_prefix": "*",
    "access": "Allow",
    "priority": 100,
    "direction": "Inbound",
}
network_client.security_rules.begin_create_or_update(resource_group, nsg_name, "Allow_SSH", security_rule).result()

# Update outbound rules to deny all
security_rule = {
    "name": "Deny_All_Outbound",
    "protocol": "*",
    "source_port_range": "*",
    "destination_port_range": "*",
    "source_address_prefix": "*",
    "destination_address_prefix": "*",
    "access": "Deny",
    "priority": 200,
    "direction": "Outbound",
}
network_client.security_rules.begin_create_or_update(resource_group, nsg_name, "Deny_All_Outbound", security_rule).result()


In [10]:
# Create Blob Storage
storage_params = {
    'sku': {'name': 'Standard_LRS'},
    'kind': 'StorageV2',
    'location': location,
    'network_acls': {'default_action': 'Deny'}
}
storage_client.storage_accounts.begin_create(resource_group_name, storage_account_name, storage_params)

In [13]:
from azure.mgmt.network.models import PrivateEndpoint, PrivateLinkServiceConnection

In [15]:
# Configuration Parameters for Private Endpoint
private_endpoint_name = 'MyPrivateEndpoint'
private_connection_resource_id = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.MachineLearningServices/workspaces/{aml_workspace_name}"
group_id = 'amlworkspace'
connection_name = 'workspace'

# Create Private Endpoint
private_endpoint_params = PrivateEndpoint(
    location=location,
    subnet={
        'id': subnet_id
    },
    private_link_service_connections=[
        PrivateLinkServiceConnection(
            name=connection_name,
            private_link_service_id=private_connection_resource_id,
            group_ids=[group_id]
        )
    ]
)

network_client.private_endpoints.begin_create_or_update(
    resource_group_name,
    private_endpoint_name,
    private_endpoint_params
)

print("Private endpoint created.")


Private endpoint created.


In [17]:
from azure.ai.ml.entities import Workspace
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    Workspace,
    ManagedNetwork,
    IsolationMode,
    ServiceTagDestination,
    PrivateEndpointDestination,
    FqdnDestination
)
from azure.identity import DefaultAzureCredential

In [ ]:


# Replace with the values for your Azure subscription and resource group.
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"

# get a handle to the subscription
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group)

In [ ]:
# # Creating a unique workspace name with current datetime to avoid conflicts
# from azure.ai.ml.entities import Workspace
# import datetime

# basic_workspace_name = "mlw-basic-prod-" + datetime.datetime.now().strftime(
#     "%Y%m%d%H%M"
# )

# ws_basic = Workspace(
#     name=basic_workspace_name,
#     location="eastus",
#     display_name="Basic workspace-example",
#     description="This example shows how to create a basic workspace",
#     hbi_workspace=False,
#     tags=dict(purpose="demo"),
# )

# ws_basic = ml_client.workspaces.begin_create(ws_basic).result()
# print(ws_basic)

In [ ]:
# Basic managed network configuration
network = ManagedNetwork(IsolationMode.ALLOW_INTERNET_OUTBOUND, )

# Workspace configuration
ws = Workspace(
    name="myworkspace",
    location="eastus",
    managed_network=network
)

# Example private endpoint outbound to a blob
rule_name = "myrule"
service_resource_id = "/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.Storage/storageAccounts/<STORAGE_ACCOUNT_NAME>"
subresource_target = "blob"
spark_enabled = True

# Add the outbound 
ws.managed_network.outbound_rules = [PrivateEndpointDestination(
    name=rule_name, 
    service_resource_id=service_resource_id, 
    subresource_target=subresource_target, 
    spark_enabled=spark_enabled)]

# Create the workspace
ws = ml_client.workspaces.begin_create(ws).result()